In [2]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


True
12.4
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [3]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import torch

# Verificar si la GPU está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando el dispositivo: {device}")

# Cargar el archivo CSV de películas
movies_df = pd.read_csv("./wiki_movie_plots_deduped.csv")

# Filtrar campos necesarios
movies_df = movies_df[movies_df['Plot'].notna()]  # Eliminar filas con Plot vacío
documents = movies_df['Plot'].tolist()  # Lista de las tramas
titles = movies_df['Title'].tolist()   # Lista de los títulos

# Configurar cliente Chroma
client = chromadb.Client()

# Crear la función de embeddings con GPU
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2",  # Modelo eficiente y rápido
    device=device  # Especificar el dispositivo (GPU o CPU)
)

# Crear colección con la función de embeddings configurada
collection = client.get_or_create_collection(
    name="movies_collection",
    embedding_function=embedding_function
)

# Generar IDs únicos para cada documento
ids = [f"movie_{i}" for i in range(len(movies_df))]

# Dividir los datos en lotes
batch_size = 5000  # Tamaño del lote (ajustar según sea necesario)
for i in range(0, len(documents), batch_size):
    batch_documents = documents[i:i+batch_size]
    batch_titles = titles[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    
    # Agregar el lote a la colección
    collection.add(
        documents=batch_documents,  # Tramas del lote
        metadatas=[{"title": title} for title in batch_titles],  # Metadata del lote
        ids=batch_ids  # IDs únicos del lote
    )
    print(f"Lote {i // batch_size + 1} procesado exitosamente.")

print(f"Se han agregado {len(documents)} documentos y generado sus embeddings.")


Usando el dispositivo: cuda
Lote 1 procesado exitosamente.
Lote 2 procesado exitosamente.
Lote 3 procesado exitosamente.
Lote 4 procesado exitosamente.
Lote 5 procesado exitosamente.
Lote 6 procesado exitosamente.
Lote 7 procesado exitosamente.
Se han agregado 34886 documentos y generado sus embeddings.


In [9]:
# Definir una consulta (puede ser una trama de película o cualquier texto)
query = "A young boy discovers "

# Realizar la consulta en la colección
results = collection.query(
    query_texts=[query],  # Pasar la consulta como texto
    n_results=5  # Número de resultados a devolver
)

# Mostrar los resultados
for i, document in enumerate(results['documents'][0]):  # Accedemos a la lista de documentos en el primer índice
    print(f"Resultado {i + 1}:")
    print(f"Trama: {document}")  # Solo mostramos la trama si no hay metadata disponible
    print("-" * 50)


Resultado 1:
Trama: An investigative thriller based on the search for a missing youngster.
--------------------------------------------------
Resultado 2:
Trama: A young boy from the lower caste resorts to petty thefts to make both ends meet after he lost his father at the early age. Once the boy realizes the importance of an education, he begins to improve his life and never looks back. Through diligence and dedication, he climbs the social and political ladder to success.
--------------------------------------------------
Resultado 3:
Trama: Thirteen-year-old Jesse is assigned a school project. A photographic self-portrait intended to portray one’s self without resorting to literal representation. Jesse lives with his parents, Sabi and Tim, in the lefty, middle class Toronto neighbourhood of Riverdale. A quiet and distant only-child with budding artistic aspirations, Jesse is inspired by the assignment to look for excitement and meaning in the world around him. Wielding a newly acqui